In [4]:
# we use this simple program to take a look at the data files 
import pickle
import pandas as pd
import torch
import numpy as np
import zipfile

In [6]:
# data paths
ruling_data_path = '/data/Dropbox/Projects/originalism/data/BloombergVOTELEVEL_Touse.dta'
sentences_data_path = '/data/Dropbox/judge_embedding_data_sp18/sentences_data.csv'
cite_graph_path = '/data/Dropbox/Data/corpora/chen-cases/cite-graph/graph.zip'
judge_bio_data_path = '/data/Dropbox/Data/Judge-Bios/judgebios/JudgesBioReshaped_TOUSE.dta'
topic_data_path = '/data/Dropbox/Projects/Ash_Chen/metadata/bb2topic.pkl'
processed_data_path = '/data/Dropbox/judge_embedding_data_sp18'

meta_data_path = '/data/Dropbox/judge_embedding_data_sp18/circuit_metadata_excerpt.dta'
table_of_cases_path = '/data/Dropbox/judge_embedding_data_sp18/tableofcases'

pd.options.display.max_columns = 999

In [16]:
zf = zipfile.ZipFile(cite_graph_path, 'r')
print(zf.namelist())

['graph.csv']


In [30]:
citation_graph_file_ptr = zf.open('graph.csv')
citation_graph_iter = pd.read_csv(citation_graph_file_ptr,iterator=True)

In [31]:
citation_graph_df = citation_graph_iter.get_chunk(20)
citation_graph_df

,year,source,sink,text
0,1880_complete,XB0P5FQNB5G0,XFJCTF,"<p class=""level"" align="""" style=""text-indent: ..."
1,1880_complete,XDOL27QNB5G0,XFJBG1,"<p class=""level"" align="""" style=""text-indent: ..."
2,1880_complete,XDOL27QNB5G0,XFJC27,"<p class=""level"" align="""" style=""text-indent: ..."
3,1880_complete,XDOL7RQNB5G0,XFJCJR,"<p class=""level"" align="""" style=""text-indent: ..."
4,1880_complete,XFIIQD,XFKDFI,"<p class=""level"" align="""" style=""text-indent: ..."
5,1880_complete,XFIJ2N,XFKR5C,"<p class=""level"" align="""" style=""text-indent: ..."
6,1880_complete,XFIJ36,XFIJ46,"<p class=""level"" align="""" style=""text-indent: ..."
7,1880_complete,XFIJ6O,XFJCBO,"<p class=""level"" align="""" style=""text-indent: ..."
8,1880_complete,XFIJ8E,XFJCFO,"<p class=""level"" align="""" style=""text-indent: ..."
9,1880_complete,XFIJ8R,XFKD7I,"<p class=""level"" align="""" style=""text-indent: ..."


In [15]:
df_ruling_iter = pd.read_stata(ruling_data_path,iterator=True) # this file is like ~6GB
df_bio = pd.read_stata(judge_bio_data_path) # this file is small
# df_citation_iter = pandas.read_stata(cite_graph_path,iterator=True)
df_ruling = df_ruling_iter.get_chunk(1000)


In [17]:
df_ruling = df_ruling_iter.get_chunk(5000)
df_ruling[['JudgeCONCURRING','JudgeDissentingTouse','JudgesListTouse']]

,caseid,Circuit,docketnumber,citation,MajOpinionWordCount,MajSelfCertainWords,minOpinionWordCount1,MinSelfCertainWords1,ConcurenceWordCount1,ConcurSelfCertainWords1,minOpinionWordCount2,MinSelfCertainWords2,ConcurenceWordCount2,ConcurSelfCertainWords2,j,date,dissentdummy,JudgeCONCURRING,JudgeDissentingTouse,songername,jOrigname,dissentdate,geniss1,geniss2,geniss3,geniss4,geniss,Affirmed,AffirmedInPart,Reversed,ReversedInPart,Vacated,VacatedInPart,Remanded,Treat,Dissenting1,Dissenting2,JudgesCount,JudgesListTouse,DissentOrConcurenceOpinionOnly,dissentOrconcurCaseid,DissentOrConcuOnlyIncluded,Judgeconcurring,JudgeDissenting,MultipleDissents,PossibleTypoInDissenterOrJudge,RehearingOrPetition,ConcurenceMistakenForDissent,FiledDate,dateamended,month,day,distance,quartertoelect,quarter,yearq,lastquarter,last3,AmendedDate,distanceAM,quartertoelectAM,JudgeDISSENTING1,JudgeDISSENTING2,Author,Writer,judgeidentificationnumber,AppointmentDate1,TerminationDate1,courtname1,courttype1,RecessAppointDate1,Circuitjudge1,AppointmentDate2,TerminationDate2,courtname2,courttype2,RecessAppointDate2,Circuitjudge2,AppointmentDate3,TerminationDate3,courtname3,courttype3,RecessAppointDate3,Circuitjudge3,AppointmentDate4,TerminationDate4,courtname4,courttype4,RecessAppointDate4,Circuitjudge4,AppointmentDate5,TerminationDate5,courtname5,courttype5,RecessAppointDate5,Circuitjudge5,AppointmentDate6,TerminationDate6,courtname6,courttype6,RecessAppointDate6,Circuitjudge6,Term,Term2,Term3,Term4,id,AppointmentDate,TerminationDate,courtname,courttype,abarating,AbaUpdate,SenateConfirmationdate,RecessAppointDate,birthday,birthmonth,birthyear,deathmonth,deathday,deathyear,JudgeAssignedCourt,judgelastname,judgefirstname,judgemiddlename,suffix,Circuitjudge,State,retirementfromactiveservice,dateaschiefjudgebegin,presidentname,seconddateaschiefjudgebegin,senatevoicevote,senatevoteayesnays,Committeeactiondate,Degree1,Degree2,Degree3,Degree4,Degree5,degreeyear1,degreeyear2,degreeyear3,degreeyear4,degreeyear5,Committeeaction,ageon,vicelastnamepredecessor,vicefirstnamepredecessor,hearings,reportnumber,authorizationlegislation,placeofbirthcity,placeofbirthstate,placeofdeathcity,placeofdeathstate,deathdate,party_Updated,dateaschiefjudgeend,seconddateaschiefjudgeend,renominatingpresidentname,partyaffiliationofrenompres,referraldatejudicialcom,degree,degree_law,degree_other,district,nominationdatesenateexecutivej,genderNew,raceorethnicity,partyaffiliationofpresident,recessappointmentdate,committeeaction,committeeactiondate,senatevotedateconfirmationdate,commissiondate,dateoftermination,terminationspecificreason,nameofschool1,degree1,nameofschool2,degree2,nameofschool3,degree3,nameofschool4,degree4,nameofschool5,degree5,employmenttextfield,bankruptcyandmagistrateservice,Nominationdate,District_Circuit,party,pres,left,StateOfResidence,ResidenceCity,appres,aba,congresi,unityi,hdem,hrep,sdem,srep,hother,sother,x_dem,x_republican,x_instate_ba,x_elev,x_unity,x_aba,x_crossa,x_pfedjdge,x_pindreg1,x_plawprof,x_pscab,x_pcab,x_pusa,x_pssenate,x_paag,x_psp,x_pslc,x_pssc,x_pshouse,x_psg,x_psgo,x_psenate,x_psatty,x_pprivate,x_pmayor,x_plocct,x_phouse,x_pgov,x_pda,x_pcc,x_pccoun,x_pausa,x_pasatty,x_pag,x_pada,x_pgovt,x_llm_sjd,x_protestant,x_evangelical,x_mainline,x_noreligion,x_catholic,x_jewish,x_black,x_nonwhite,x_female,x_jd_public,x_ba_public,x_b10s,x_b20s,x_b30s,x_b40s,x_b50s,x_pbank,x_pmag,SchoolState1,Public1,Private1,SchoolState2,Public2,Private2,SchoolState3,Public3,Private3,SchoolState4,Public4,Private4,SchoolState5,Public5,Private5,TonotInclude,idtocorrrect,ayear,x_ageon40s,x_ageon50s,x_ageon60s,x_ageon40orless,x_ageon70ormore,pres_correct2,political_party,x_pago,startdate,Mergebio,dissentvote,JudgesfromThecircuit,visiting,BecameSenior,Senior,TypeOfCourt,reversedummy,reverseandremand,reversenoremand,affirmdummy,ww1,ww1half1,ww1half2,ww2,ww2half1,ww2half2,korean,koreanhalf1,koreanhalf2,vietnam,vietnamhalf1,vietnamhalf2,gulf,gulfhalf1,gulfhalf2,afghan,Iraq,Iraqhalf1,Iraqhalf2,war,w

In [18]:
df_ruling[['caseid','JudgeCONCURRING','JudgeDissentingTouse','JudgesListTouse']]

,JudgeCONCURRING,JudgeDissentingTouse,JudgesListTouse
1000,,,WILKINS | TRAXLER | WILLIAMS
1001,,,WILKINS | TRAXLER | WILLIAMS
1002,SHEDD,LUTTIG,LUTTIG | SHEDD | KING
1003,SHEDD,LUTTIG,LUTTIG | SHEDD | KING
1004,SHEDD,LUTTIG,LUTTIG | SHEDD | KING
1005,,,NEWMAN | CABRANES | KEARSE
1006,,,NEWMAN | CABRANES | KEARSE
1007,,,NEWMAN | CABRANES | KEARSE
1008,,,FLAUM | WOOD | EASTERBROOK
1009,,,FLAUM | WOOD | EASTERBROOK


In [9]:
df_bio

,songername,judgeidentificationnumber,Term,id,AppointmentDate,TerminationDate,courtname,courttype,abarating,AbaUpdate,SenateConfirmationdate,RecessAppointDate,birthday,birthmonth,birthyear,deathmonth,deathday,deathyear,JudgeAssignedCourt,judgelastname,judgefirstname,judgemiddlename,suffix,Circuitjudge,SeatState,retirementfromactiveservice,dateaschiefjudgebegin,presidentname,seconddateaschiefjudgebegin,senatevoicevote,senatevoteayesnays,Committeeactiondate,Degree1,Degree2,Degree3,Degree4,Degree5,degreeyear1,degreeyear2,degreeyear3,degreeyear4,degreeyear5,Committeeaction,ageon,vicelastnamepredecessor,vicefirstnamepredecessor,hearings,reportnumber,authorizationlegislation,placeofbirthcity,placeofbirthstate,placeofdeathcity,placeofdeathstate,deathdate,party_Updated,dateaschiefjudgeend,seconddateaschiefjudgeend,renominatingpresidentname,partyaffiliationofrenompres,referraldatejudicialcom,degree,degree_law,degree_other,district,nominationdatesenateexecutivej,genderNew,raceorethnicity,partyaffiliationofpresident,recessappointmentdate,committeeaction,committeeactiondate,senatevotedateconfirmationdate,commissiondate,dateoftermination,terminationspecificreason,nameofschool1,degree1,nameofschool2,degree2,nameofschool3,degree3,nameofschool4,degree4,nameofschool5,degree5,employmenttextfield,bankruptcyandmagistrateservice,Nominationdate,District_Circuit,party,pres,left,state,city,appres,aba,congresi,unityi,hdem,hrep,sdem,srep,hother,sother,x_dem,x_republican,x_instate_ba,x_elev,x_unity,x_aba,x_crossa,x_pfedjdge,x_pindreg1,x_plawprof,x_pscab,x_pcab,x_pusa,x_pssenate,x_paag,x_psp,x_pslc,x_pssc,x_pshouse,x_psg,x_psgo,x_psenate,x_psatty,x_pprivate,x_pmayor,x_plocct,x_phouse,x_pgov,x_pda,x_pcc,x_pccoun,x_pausa,x_pasatty,x_pag,x_pada,x_pgovt,x_llm_sjd,x_protestant,x_evangelical,x_mainline,x_noreligion,x_catholic,x_jewish,x_black,x_nonwhite,x_female,x_jd_public,x_ba_public,x_b10s,x_b20s,x_b30s,x_b40s,x_b50s,x_pbank,x_pmag,SchoolState1,Public1,Private1,SchoolState2,Public2,Private2,SchoolState3,Public3,Private3,SchoolState4,Public4,Private4,SchoolState5,Public5,Private5,TonotInclude,idtocorrrect,ayear,x_ageon40s,x_ageon50s,x_ageon60s,x_ageon40orless,x_ageon70ormore,pres_correct2,political_party,x_pago,startdate
0,"ABRAMS, RONNIE",3419,1,NaN,2012-03-23,NaT,"u. s. district court, southern district of new...",usdc,Qualified,,2012-03-22,NaT,NaN,NaN,1968.0,NaN,NaN,NaN,,ABRAMS,RONNIE,,,2,NY,,NaN,Barack Obama,NaN,No,96/2,2011-11-03,2.0,5.0,NaN,NaN,NaN,1990,1993,,,NaN,1.0,44.0,Kaplan,Lewis A.,10/04/2011,,,New York,NY,,,NaT,NaN,NaN,NaN,,,07/28/2011,,,,0208,07/28/2011,F,White,Democratic,,Reported With Recommendation,11/03/2011,03/22/2012,03/23/2012,,,Cornell University,B.A.,Yale Law School,J.D.,,,,,,,"Law clerk, Hon. Thomas P. Griesa, U.S. Distric...",,18836.0,,NaN,94.0,NaN,NaN,NaN,1.0,3.0,112TH,2.0,193.0,242.0,52,47,0.0,4.0,1.0,0.0,1.0,NaN,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NY,0.0,1.0,CT,0.0,1.0,,NaN,NaN,,NaN,NaN,,NaN,NaN,0.0,NaN,2012.0,1.0,0.0,0.0,0.0,0.0,NaN,,0.0,2012-03-23
1,"ABRUZZO, MATTHEW J.",1,1,2.0,1936-02-15,1971-05-28,"u. s. district court, eastern district of new ...",usdc,,,1936-02-12,NaT,30.0,4.0,1889.0,5.0,28.0,1971.0,Brooklyn,ABRUZZO,MATTHEW,T.,,2,NY,02/15/1966,NaN,Franklin D. Roosevelt,NaN,Yes,,1936-02-11,6.0,NaN,NaN,NaN,NaN,1910,,,,NaN,1.0,47.0,new,,,,49 Stat. 659,Brooklyn,NY,Potomac,MD,1971-05-28,1.0,NaN,NaN,,,02/03/1936,,,,0207,02/03/1936,M,White,Democratic,,Reported With Recommendation,02/11/1936,02/12/1936,02/15/1936,05/28/1971,Death,Brooklyn Law School,LL.B.,,,,,,,,,"Private practice, Brooklyn, New York, 1910-1936",,-8733.0,0207,1.0,14.0,3.0,32.0,3220.0,1.0,6.0,74TH,0.0,322.0,103.0,69,25,0.0,2.0,1.0,0.0,0.0,NaN,1.0,0.0,0.0,NaN,1.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,

In [10]:
df_bio[['songername','party','appres','presidentname','party_Updated',]]

,songername,party,appres,presidentname,party_Updated
0,"ABRAMS, RONNIE",NaN,1.0,Barack Obama,NaN
1,"ABRUZZO, MATTHEW J.",1.0,1.0,Franklin D. Roosevelt,1.0
2,"ACHESON, MARCUS W.",0.0,0.0,Rutherford B. Hayes,0.0
3,"ACHESON, MARCUS W.",0.0,0.0,Benjamin Harrison,0.0
4,"ACKER, WILLIAM MARSH, JR.",0.0,0.0,Ronald Reagan,0.0
5,"ACKERMAN, HAROLD ARNOLD",1.0,1.0,Jimmy Carter,1.0
6,"ACKERMAN, JAMES WALDO",0.0,0.0,Gerald Ford,0.0
7,"ACKERMAN, JAMES WALDO",0.0,0.0,Gerald Ford,0.0
8,"ACOSTA, RAYMOND L.",0.0,0.0,Ronald Reagan,0.0
9,"ADAIR, JEFFERSON LEROY",1.0,1.0,Franklin D. Roosevelt,1.0


In [48]:
df_sentences_ptr = pd.read_csv(sentences_data_path,iterator=True)
df_sentences = df_sentences_ptr.get_chunk(20)
df_sentences

,Unnamed: 0,caseid,year,judge_last_name,middle_part,opinion_text
0,0,XB0PMNQNB5G0,1891,PARDEE,contentMajOp,"P ARDEE , J., ( after stating the case .)\nTh..."
1,1,XFL757,1891,BROWN,contentMajOp,"B ROWN , Circuit Justice, ( orally .)\nIn thi..."
2,2,XB0PMRQNB5G0,1891,PARDEE,contentMajOp,"P ARDEE , J. The case on this intervention is..."
3,3,XFL7KI,1891,LOCKE,contentMajOp,"L OCKE , District Judge.\nThis motion coming ..."
4,4,XB0OI7QNB5G0,1891,NELSON,contentMajOp,"N ELSON , J. This was an action brought to re..."
5,5,XB0PHFQNB5G0,1891,PARDEE,contentMajOp,"P ARDEE , J. The record shows a suit brought ..."
6,6,XB0PGVQNB5G0,1891,THAYER,contentMajOp,"T HAYER , J., ( after stating the facts as ab..."
7,7,XB0PJRQNB5G0,1891,BRUCE,contentMajOp,"B RUCE , J. This is a suit brought in the cou..."
8,8,XFL6PE,1891,BROWN,contentMajOp,"B ROWN , Circuit Justice, ( orally .)\nIn thi..."
9,9,XB0P8NQNB5G0,1891,HALLETT,contentDisOp,"H ALLETT , J., dissents.\n"


In [42]:
topic_data = pickle.load(open(topic_data_path,"rb"))

In [43]:
topic_data

{'X3TQ24': 'Tax & Accounting',
 'XABJ9T': 'Transportation Law',
 'X1CGD3M003': 'Constitutional Law',
 'X2O2CO': 'Securities Law',
 'XFIJ6F': 'Bankruptcy Law',
 'X3RVMJ': '',
 'X3847P': 'Civil Procedure',
 'X12N8D0003': 'Appellate Procedure',
 'X1DHN30003': 'Appellate Procedure',
 'X468JP': 'Civil Procedure',
 'X39MDQ': 'Medical Malpractice',
 'X550MK': 'Civil Rights',
 'X3I1DB': 'Privacy & Information Law',
 'X45OJ5': 'Civil Procedure',
 'X3J8L1': 'Criminal Law',
 'X50T9E': 'Criminal Law',
 'XFV9F0': 'Criminal Law',
 'X3NQNP': 'Criminal Law',
 'X41ESH': 'Tax & Accounting',
 'X36KAI': 'Criminal Law',
 'X3T9S2': 'Criminal Law',
 'X3FEB2': "Prisoners' Rights",
 'X3AEMS': 'Social Security',
 'X3IQJQ': 'Bankruptcy Law',
 'X3FG7H': 'Government Employees',
 'X43BU9': 'Criminal Law',
 'X45PF6': 'Damages & Remedies',
 'X3RNF3': 'Damages & Remedies',
 'X1I5JBM003': 'Civil Procedure',
 'X21KRG': 'Civil Procedure',
 'XFP2I4': 'Constitutional Law',
 'XAFL9O': 'Criminal Law',
 'X3OSS3': 'Damages & R

In [8]:
#meta data 
df_meta_ptr = pd.read_stata(meta_data_path,iterator=True)
df_meta = df_meta_ptr.get_chunk(2000)
df_meta



,caseid,Circuit,j,date,songername,Affirmed,AffirmedInPart,Reversed,ReversedInPart,Vacated,VacatedInPart,Remanded,Dissenting1,Author,Writer
0,X101L14003,4,2,2004-03-24,"DUNCAN, ALLYSON KAY",0,0,0,0,0,0,0,0,MOTZ,1.0
1,X101L14003,4,3,2004-03-24,"KING, ROBERT BRUCE",0,0,0,0,0,0,0,0,MOTZ,1.0
2,X101L14003,4,1,2004-03-24,"MOTZ, DIANA GRIBBON",0,0,0,0,0,0,0,0,MOTZ,1.0
3,X10284I003,4,3,2004-03-31,"LUTTIG, J. MICHAEL",0,1,0,0,0,1,1,0,WILKINSON,1.0
4,X10284I003,4,1,2004-03-31,"WILKINSON, J. HARVIE, III",0,1,0,0,0,1,1,0,WILKINSON,1.0
5,X10284I003,4,2,2004-03-31,"TRAXLER, WILLIAM BYRD, JR.",0,1,0,0,0,1,1,0,WILKINSON,1.0
6,X102UPO003,3,3,2004-04-05,"RENDELL, MARJORIE O.",0,0,0,0,1,0,1,0,RENDELL,3.0
7,X102UPO003,3,1,2004-04-05,"SLOVITER, DOLORES K.",0,0,0,0,1,0,1,0,RENDELL,3.0
8,X102UPO003,3,2,2004-04-05,"ALDISERT, RUGGERO",0,0,0,0,1,0,1,0,RENDELL,3.0
9,X103660N,9,1,1994-07-25,"HUG, PROCTER R., JR.",1,0,0,0,0,0,0,0,,NaN
